<a href="https://colab.research.google.com/github/cvetyshayasiren/colabs/blob/main/Pokemon_G0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title <- Запуск

!pip install wget
!pip install ipyleaflet
!pip install gTTS

import ipywidgets as widgets
from ipywidgets import Layout, ButtonStyle
import wget
import os
import pandas as pd
import time
import re
from random import choice, uniform, choices
from ipyleaflet import Map, Marker, Icon
from gtts import gTTS
from IPython.display import Audio
from IPython.display import display
from IPython.display import clear_output
import requests
from collections import namedtuple
from functools import wraps

clear_output(wait=True)

def singleton(cls):
    """Декоратор синглтон для классов"""
    @wraps(cls)
    def wrapper(*args, **kwargs):
        if not wrapper.instance:
            wrapper.instance = cls(*args, **kwargs)
        return wrapper.instance
    wrapper.instance = None
    return wrapper

@singleton
class Strings():
  def __init__(self):
    self.progress_description = 'Загрузка:'
    self.state_common_status = ''
    self.state_caught_status = 'Пойман ✔️'
    self.state_run_status = 'убежал ✖️'
    self.state_egg_finish_status = 'Яйцо 🥚'
    self.state_egg_status = 'В яйце 🥚'
    self.state_egg_short_text = 'Это ваш {}, жмите скорее чтобы узнать кто это 🥚'
    self.state_egg_full_text = 'Ваш {} сидит в яйце 🥚. Тыкните на иконку на карт, чтобы узнать кто это.'
    self.state_raid_finish_status = 'Рейд 👾'
    self.state_raid_status = 'В рейде 👾'
    self.state_raid_short_text = 'Рейд, {}. Жмите чтобы завершить и поймать.'
    self.state_raid_full_text = '{} Находится в рейде. Найдите его на карте, а затем нажмите, чтобы узнать кто это.'
    self.pokemon_shiny = '✨Шайни '
    self.combobox_placeholder = 'Выберите Покемона'
    self.textarea = 'Выберите покемона из списка или испытайте удачу.'

  def get_random_cograt(self, pokemon):
    state_name = pokemon.state['name']
    category = pokemon.category
    end = f'{pokemon.name_ru}. {pokemon.description}' if state_name not in ['egg', 'raid'] else ''
    
    l = ['common', 'caught', 'run', 'egg_finish', 'egg', 'raid_finish', 'raid']
    congrat = {
        'common': ['', 'Эй, это', 'Хаха, не буду говорить. Бебебебе. Ладно'],
        'caught': ['Ой, лол, да вы поймали', 'Поздравляю с поимкой', 'Вам достался', 
                   'Хаха, это же', 'Оу, у вас', 'На вашу голову свалился'],
        'run': ['Вы упустили', 'От вас слинял', 'Вы смотрите в след', 'От вас убежал', 
                'Спародировал колобка'],
        'egg_finish': ['В яйце оказался', 'Из яйца выпрыгнул', 'Вылупился', 'Вышел на свободу'],
        'egg': ['Ушлёпок сидит в яйце. Вам надо найти его на карте и жмякнуть.', 
                f'У вас {category}, но он в яйце', f'Вы нашли яйцо. Там {category}'],
        'raid_finish': ['После сложной битвы, вам достался', 'Закончился рейд. Вы поймали', 'Зарейжен'],
        'raid': ['Вы наткнулись на рейд', 'Это рейд. Найдите его на карте, нажмите и узнаете кто там.', 
                 f'Это {category}, но он в рейде', f'{category}, заверши рейд чтобы стало понятнее.']
    }
    return choice(congrat[state_name]) + end

@singleton
class Urls():
  def __init__(self):
    # self.logo = 'https://raw.githubusercontent.com/PokeMiners/pogo_assets/master/Images/Logos/go_logo.png'
    self.logo = 'https://raw.githubusercontent.com/PokeMiners/pogo_assets/master/Images/Logos/pokemongo_logo_white_outline.png'
    self.json = 'https://raw.githubusercontent.com/PokeMiners/game_masters/master/latest/latest.json'
    self.txt_russian = 'https://raw.githubusercontent.com/PokeMiners/pogo_assets/master/Texts/Latest APK/Russian.txt'
    self.txt_english = 'https://raw.githubusercontent.com/PokeMiners/pogo_assets/master/Texts/Latest APK/English.txt'
    self.icon_unknown = 'https://raw.githubusercontent.com/PokeMiners/pogo_assets/master/Images/Stickers/sticker_lentilLaunch_camera.png'
    self.icon_egg = 'https://raw.githubusercontent.com/PokeMiners/pogo_assets/master/Images/Stickers/sticker_goTour2022_togepi.png'
    self.icon_raid = 'https://raw.githubusercontent.com/PokeMiners/pogo_assets/master/Images/Combat/premierball_sprite.png'
    self.url_for_check = 'https://raw.githubusercontent.com/PokeMiners/pogo_assets/master/Images/Pokemon - 256x256/pokemon_icon_{}_00.png'

  def check_url(self, url: str):
    try:
      check = requests.head(url)
      return True if check.status_code == 200 else False
    except:
      return False

  def get_pokemon_icon_url(self, dex = '001', shiny = 1):
    url = f'https://raw.githubusercontent.com/PokeMiners/pogo_assets/master/Images/Pokemon - 256x256/pokemon_icon_{dex}_00{shiny*"_shiny"}.png'
    reserve_url = self.icon_unknown
    check = self.check_url(url)
    return url if check else reserve_url

@singleton
class Data():
  def __init__(self):
    self.load()
    self.df = self.parse_json()
    self.russian, self.english = self.dicts()
    self.df_addition()

  def load(self):
    '''Скачивание json с github Покемайнеров'''
    if 'gamemaster.json' not in os.listdir():
        url = Urls().json
        wget.download(url, 'gamemaster.json')

  def progress(self, sequence, every=1):
      progress = widgets.IntProgress(
          min=0, 
          max=len(sequence)-1, 
          value=0, 
          style={'bar_color': 'green'}, 
          description=Strings().progress_description
          )
      display(progress)
      for index, record in enumerate(sequence):
          if index % every == 0:
              progress.value = index
          time.sleep(0.2)
          yield record
      time.sleep(0.2)
      progress.close()

  def dicts(self):
    russian, english = {}, {}
    urls = [[Urls().txt_russian, russian], [Urls().txt_english, english]]
    for url, dictionary in urls:
      name = os.path.basename(url)
      if not os.path.exists(name):
        wget.download(url, name)
      list_of_data = open(name).read().split('\n\n')
      for i in list_of_data:
        id = re.findall(r'RESOURCE ID: (.*)', i)
        text = re.findall(r'TEXT: (.*)', i)
        if id and text:
          dictionary[id[0]] = text[0]
    return russian, english

  def parse_json(self) -> pd.core.frame.DataFrame:
      df = pd.DataFrame()
      json = pd.read_json("gamemaster.json")
      data = json.data.apply(lambda x: {**x.get("pokemonSettings"), **{"id": x.get("templateId")}} if x.get("pokemonSettings") else None).dropna()
      apply = {
          'pokemon': lambda x: x['pokemonId'].capitalize(),
          'dex': lambda x: '{:03}'.format(int(re.findall('V0*(\d+)_', x['id'])[0])),
          'type': lambda x: f"{x['type']}/{x['type2']}".replace('POKEMON_TYPE_', '').lower() if x.get('type2') else x['type'].split('_')[-1].lower(),
          'att': lambda x: int(x['stats'].get('baseAttack', 0)),
          'def': lambda x: int(x['stats'].get('baseDefense', 0)),
          'hp': lambda x: int(x['stats'].get('baseStamina', 0))
          }
      for i in self.progress(apply):
        df[i] = data.apply(apply[i])
      df = df.drop_duplicates(subset=['pokemon'])
      df.to_csv('gamemaster.csv')
      return df

  def df_addition(self):
    self.df['name_ru'] = [self.russian[f'pokemon_name_0{dex}'] for dex in self.df['dex']]
    self.df['description'] = [self.russian[f"pokemon_desc_0{dex}"] for dex in self.df['dex']]
    self.df['category'] = [self.russian[f"pokemon_category_0{dex}"] for dex in self.df['dex']]

data = Data() # так сделано, чтобы был доступ к данным и из "командной строки"

@singleton
class State():
  def __init__(self):
    s = Strings()
    self.common = {'name': 'common', 'status': s.state_common_status}
    self.caught = {'name': 'caught', 'status': s.state_caught_status}
    self.run = {'name': 'run', 'status': s.state_run_status}
    self.egg_finish = {'name': 'egg_finish', 'status': s.state_egg_finish_status}
    self.egg = {'name': 'egg', 'next_state': self.egg_finish, 'status': s.state_egg_status, 'iv': range(5, 16), 'shiny': 3, 
                'pic_url': Urls().icon_egg, 'short_text': s.state_egg_short_text, 'full_text': s.state_egg_full_text}
    self.raid_finish = {'name': 'raid_finish', 'status': s.state_raid_finish_status}
    self.raid = {'name': 'raid', 'next_state': self.raid_finish, 'status': s.state_raid_status, 'iv': range(12, 16), 
                 'shiny': 4, 'pic_url': Urls().icon_raid, 'short_text': s.state_raid_short_text, 
                 'full_text': s.state_raid_full_text}

  def random_state(self):
    return choices([self.caught, self.run, self.egg, self.raid], weights=[10, 5, 5, 5])[0]

class Pokemon():
  def __init__(self, name = None, state = None):
    if name: df = data.df[data.df["pokemon"] == name]
    else:
      check = False
      while not check:
        df = data.df.sample()
        dex = df['dex'].values[0]
        check = Urls().check_url(Urls().url_for_check.format(dex))

    self.state = State().common if name else State().random_state()
    self.status = self.state['status']
    self.name = df['pokemon'].values[0]
    self.dex = df['dex'].values[0]
    self.attack = df['att'].values[0]
    self.defence = df['def'].values[0]
    self.hp = df['hp'].values[0]
    self.description = df['description'].values[0]
    self.category = df['category'].values[0]
    self.name_ru = df['name_ru'].values[0]
    iv = lambda: choice(self.state.get('iv', range(16)))
    self.iv = f'{iv()}-{iv()}-{iv()}'
    chance = self.state.get('shiny', 2)
    self.shiny = choices([1, 0], weights=[1/chance, 1 - 1/chance])[0]
    self.pic_url = None
    self.pic_url_name = None
    self.short_text = None
    self.full_text = None
    self.get_changable_data()

  def get_changable_data(self):
    self.pic_url = self.state.get('pic_url', Urls().get_pokemon_icon_url(self.dex, self.shiny))
    self.pic_url_name = os.path.basename(self.pic_url)
    self.short_text = self.state.get('short_text', self.get_text('short')).format(self.category)
    self.full_text = self.state.get('full_text', self.get_text('full')).format(self.category)

  def change_state(self):
    if self.state.get('next_state'):
      self.state = self.state.get('next_state')
      self.status = self.state['status']
      self.get_changable_data()

  def get_text(self, format = None):
    short = f"{self.shiny * Strings().pokemon_shiny}{self.name}/{self.name_ru}\n" \
            f"{self.category}\nIV = {self.iv}\n" \
            f"👊{self.attack} 🛡{self.defence} ➕{self.hp}\n" \
            f"{self.status}"
    if format == 'short': return short
    full = f"{short}\n\n{self.description}"
    if format == 'full': return full
    return (short, full)

  def __str__(self):
    return f'{self.full_text}\n\n{self.pic_url}'

class MyMap():
  def __init__(self, center = (59.948081, 30.466340), zoom = 11):
    self.map = Map(center=center, 
                   zoom=zoom, 
                   layout = widgets.Layout(height='800px'), 
                   scroll_wheel_zoom=True)

  def add_random_marker(self, 
                        context = None,
                        pokemon = Pokemon(),
                        lat_range = (59.873825, 60.092289), 
                        lon_range = (30.175163, 30.477287)):
    lat, lon = uniform(*lat_range), uniform(*lon_range)
    icon = Icon(icon_url=pokemon.pic_url, icon_size=[92, 92])
    marker = Marker(location=(lat, lon), 
                    draggable=False, 
                    title=pokemon.short_text, 
                    alt=pokemon.name_ru, 
                    icon = icon)
    if context: marker.on_click(self.marker_on_click(context, pokemon, marker))
    self.map.add_layer(marker)

  def marker_on_click(self, context, pokemon, marker, **kwargs):
    def on_click(**kwargs):
      if pokemon.state.get('next_state'):
        pokemon.change_state()
        self.map.remove_layer(marker)
        marker.title = pokemon.short_text
        marker.icon = Icon(icon_url=pokemon.pic_url, icon_size=[92, 92])
        self.map.add_layer(marker)
      context.show_pokemon(pokemon, new_marker=False)
    return on_click

class Ui():
  '''интерфейс пользователя'''
  def __init__(self):
    self.btn_go = widgets.Button(description='Go 🎲', button_style='success', 
                                 style = ButtonStyle(button_color='#FF6347'))
    self.btn_go.on_click(self.btn_go_clicked)
    self.combobox = widgets.Combobox(placeholder=Strings().combobox_placeholder, 
                                     options=sorted(list(data.df['pokemon'])), 
                                     ensure_option=True, 
                                     disabled=False)
    self.combobox.observe(self.combobox_observe)
    self.pic = widgets.Image(format='png', layout=Layout(width='256px', heigth='256px'))
    self.show_pic()
    self.text = widgets.Textarea(value=Strings().textarea, rows=30)
    self.checkbox_sound = widgets.Checkbox(description='🔇', layout=Layout(width='50%'))
    self.checkbox_sound.observe(self.sound_checkbox_observe)
    self.my_map = MyMap()
    self.display()

  def display(self):
    items = [widgets.HBox([widgets.VBox([widgets.HBox([self.btn_go, self.checkbox_sound]), self.combobox, self.pic, self.text]), self.my_map.map])]
    display(*items)
    
  def btn_go_clicked(self, b):
    self.show_pokemon(Pokemon())
    self.combobox.value = ''

  def sound_checkbox_observe(self, c):
    if c.name == 'value':
      self.checkbox_sound.description = '🔊' if c.new else '🔇'
      if not c.new: clear_output(wait=True); self.display()

  def combobox_observe(self, o):
    if all([o.name == 'value', o.new != '']):
      self.show_pokemon(Pokemon(name = self.combobox.value), new_marker = False)

  def show_pokemon(self, pokemon, new_marker = True):
    self.download_pic(pokemon.pic_url)
    self.show_pic(pokemon.pic_url_name)
    self.text.value = pokemon.full_text
    if new_marker: self.my_map.add_random_marker(context = self, pokemon = pokemon)
    if self.checkbox_sound.value: self.speech(text = Strings().get_random_cograt(pokemon))

  def download_pic(self, url = None):
    if not url: url = Urls().logo
    name = os.path.basename(url)
    if os.path.exists(name): return
    try:
        wget.download(url, name)
    except Exception as error:
        print(error)

  def show_pic(self, name = None):
    if not name: self.download_pic(); name = os.path.basename(Urls().logo)
    self.pic.set_value_from_file(name)

  def speech(self, text: str = 'hello', name: str = 'speech', lang: str = 'ru'):
    tts = gTTS(text, lang=lang)
    tts.save(f'{name}.mp3')
    sound = Audio(f'{name}.mp3', autoplay=True)
    clear_output(wait=True)
    self.display()
    display(sound)

clear_output(wait=True)
ui = Ui()